In [ ]:
import warnings

import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets

from ggmax_sampler import get_best_points_for_ggmax_sampling

# An efficient algorithm for sampling the shear-modulus reduction curve

The first figures shows the $G/G_{max}$ curve, and its linear interpolation with the classical *Log sampling* method vs. the *Auto sampling* method.

The second figures shows the absolute differences between the original $G/G_{max}$ curve and the linear interpolations with both methods.

You can change the number of interpolation points, or the $\gamma_{ref}$ parameter of the $G/G_{max}$ curve.

In [ ]:
X_MIN, X_MAX = 1e-7, 1e0


@widgets.interact(
    number_of_points=widgets.IntSlider(
        value=20,
        min=3,
        max=200,
        step=1,
        continuous_update=False,
    ),
    gamma_ref=widgets.FloatLogSlider(
        value=1e-4,
        base=10,
        min=np.log10(X_MIN) + 1,  # max exponent of base
        max=np.log10(X_MAX) - 1,  # min exponent of base
        step=1,  # exponent step,
        continuous_update=False,
    ),
)
def show_sampling(number_of_points, gamma_ref):
    def ggmax(e):
        return 1 / (1 + e / gamma_ref)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        # on some cases, we may divide by zero, and numpy warns us
        # about that. Let's hide this warning to the users.
        auto_sampling = get_best_points_for_ggmax_sampling(
            ggmax, X_MIN, X_MAX, number_of_points
        )
    log_sampling = np.logspace(np.log10(X_MIN), np.log10(X_MAX), number_of_points)
    epsilons = np.logspace(np.log10(X_MIN), np.log10(X_MAX), 1000)

    fig = go.Figure()
    fig.add_traces(
        [
            go.Scatter(
                x=epsilons,
                y=ggmax(epsilons),
                name="$G/G_{max}$",
                mode="lines",
                line=dict(color="#00CC96"),
            ),
            go.Scatter(
                x=log_sampling,
                y=ggmax(log_sampling),
                name="Log sampling",
                mode="markers+lines",
                line=dict(color="#636EFA"),
                marker_symbol="x",
            ),
            go.Scatter(
                x=log_sampling,
                y=np.zeros_like(log_sampling),
                showlegend=False,
                mode="markers",
                line=dict(color="#636EFA"),
                marker_symbol="x",
            ),
            go.Scatter(
                x=auto_sampling,
                y=ggmax(auto_sampling),
                name="Auto sampling",
                mode="markers+lines",
                line=dict(color="#EF553B"),
            ),
            go.Scatter(
                x=auto_sampling,
                y=np.zeros_like(auto_sampling),
                showlegend=False,
                mode="markers",
                line=dict(color="#EF553B"),
            ),
        ]
    )
    fig.update_xaxes(
        type="log",
    )

    fig.update_layout(
        xaxis_title=r"$\varepsilon$",
        yaxis_title=r"$G/G_{max}$",
        title=f"Sampling of shear-modulus reduction curve with {number_of_points} points",
        height=500,
    )
    display(fig)

    fig = go.Figure()
    fig.add_traces(
        [
            go.Scatter(
                x=epsilons,
                y=abs(
                    ggmax(epsilons)
                    - np.interp(epsilons, log_sampling, ggmax(log_sampling))
                ),
                name="Log sampling",
                mode="lines",
                line=dict(color="#636EFA"),
            ),
            go.Scatter(
                x=epsilons,
                y=abs(
                    ggmax(epsilons)
                    - np.interp(epsilons, auto_sampling, ggmax(auto_sampling))
                ),
                name="Auto sampling",
                mode="lines",
                line=dict(color="#EF553B"),
            ),
        ]
    )
    fig.update_xaxes(
        title_text=r"$\varepsilon$",
        type="log",
    )
    fig.update_yaxes(
        title_text=r"Difference",
        type="log",
    )
    return fig